### _Yue You_

Load longdata.RData from bCourses. You may use Odat.long or Odat.wide for your analysis. They are identical except for the format. The columns are labeled according to how we normally notate them. A1's are the binary treatments, A2's are the censoring indicators, Y's are the binary survival outcomes and L's  are the covariates, which are either continuous or binary as you will see upon looking at the dataframe. Fortunately, the data is already cleaned, as in, NA's are produced for all nodes after outcome is 1 (death) and also after censoring occurs.  

### 1

Assuming sequential randomization and positivity as usual, estimate the difference in counterfactual mean outcomes at time 5 under treatment and not: $\mathbb{E}[Y(5)_{\bar{a}=(0,0,1,1,1)}]-\mathbb{E}[Y(5)_{\bar{a}=(0,0,0,0,0)}]$. Give estimates using IPTW and TMLE (debiasing eachh outcome regression with a targeting step, moving backward in time and otherwise known as L-TMLE) and show all code used to do so.  Your superlearner library should include three algorithms, one of each of the following: glm, mean, xgboost. Provide instructions on how I may access your superlearner results in this estimation process.

In [2]:
options(warn=-1)
rm(list=ls())
setwd("~/Desktop/ph240b-survival_analysis/HW4/Yue")
load("longdata.RData")
library(survival)
library(survminer)
library(ltmle)
library(SuperLearner)
library(xgboost)

In [3]:
Odat = Odat.wide[, c(2:5, 7:27)]
Anodes = grep("A1", colnames((Odat)))
Cnodes = grep("A2", colnames((Odat)))
Lnodes = grep("L", colnames((Odat)))
Ynodes = grep("Y", colnames((Odat)))
for (col in Cnodes) {
  Odat[, col] = BinaryToCensoring(is.censored = Odat[col])
} 
Odat[, "Y_1"][is.na(Odat$Y_1)] = 1
Odat[, "Y_2"][is.na(Odat$Y_2)] = 1
Odat[, "Y_3"][is.na(Odat$Y_3)] = 1
Odat[, "Y_4"][is.na(Odat$Y_4)] = 1
Odat[, "Y_5"][is.na(Odat$Y_5)] = 1

In [4]:
invisible(results.ltmle <- ltmle(Odat, 
                       Anodes = Anodes, 
                       Cnodes = Cnodes, 
                       Lnodes = Lnodes, 
                       Ynodes = Ynodes, 
                       abar = list(c(0, 0, 1, 1, 1), c(0, 0, 0, 0, 0)), 
                       survivalOutcome = TRUE, 
                       SL.library = c("SL.mean", "SL.glm", "SL.xgboost"), 
                       variance.method = "ic"))


Note: for internal purposes, all nodes after a censoring event are set to NA and 
 all nodes (except Ynodes) are set to NA after Y=1 if survivalFunction is TRUE (or if specified by deterministic.Q.function).
 Your data did not conform and has been adjusted. This may be relevant if you are 
 writing your own deterministic function(s) or debugging ltmle.
Note: for internal purposes, all nodes after a censoring event are set to NA and 
 all nodes (except Ynodes) are set to NA after Y=1 if survivalFunction is TRUE (or if specified by deterministic.Q.function).
 Your data did not conform and has been adjusted. This may be relevant if you are 
 writing your own deterministic function(s) or debugging ltmle.
Note: for internal purposes, all nodes after a censoring event are set to NA and 
 all nodes (except Ynodes) are set to NA after Y=1 if survivalFunction is TRUE (or if specified by deterministic.Q.function).
 Your data did not conform and has been adjusted. This may be relevant if you are 

The IPTW results are:

In [5]:
summary(results.ltmle, "iptw")

Estimator:  iptw 
Call:
ltmle(data = Odat, Anodes = Anodes, Cnodes = Cnodes, Lnodes = Lnodes, 
    Ynodes = Ynodes, survivalOutcome = TRUE, abar = list(c(0, 
        0, 1, 1, 1), c(0, 0, 0, 0, 0)), SL.library = c("SL.mean", 
        "SL.glm", "SL.xgboost"), variance.method = "ic")

Treatment Estimate:
   Parameter Estimate:  0.58667 
    Estimated Std Err:  0.020547 
              p-value:  <2e-16 
    95% Conf Interval: (0.5464, 0.62694) 

Control Estimate:
   Parameter Estimate:  0.60767 
    Estimated Std Err:  0.032829 
              p-value:  <2e-16 
    95% Conf Interval: (0.54333, 0.67202) 

Additive Treatment Effect:
   Parameter Estimate:  -0.021002 
    Estimated Std Err:  0.038339 
              p-value:  0.58383 
    95% Conf Interval: (-0.096144, 0.05414) 

Relative Risk:
   Parameter Estimate:  0.96544 
  Est Std Err log(RR):  0.063725 
              p-value:  0.58099 
    95% Conf Interval: (0.85208, 1.0939) 

Odds Ratio:
   Parameter Estimate:  0.91638 
  Est Std Err lo

The TMLE results are:

In [6]:
summary(results.ltmle, "tmle")

Estimator:  tmle 
Call:
ltmle(data = Odat, Anodes = Anodes, Cnodes = Cnodes, Lnodes = Lnodes, 
    Ynodes = Ynodes, survivalOutcome = TRUE, abar = list(c(0, 
        0, 1, 1, 1), c(0, 0, 0, 0, 0)), SL.library = c("SL.mean", 
        "SL.glm", "SL.xgboost"), variance.method = "ic")

Treatment Estimate:
   Parameter Estimate:  0.58088 
    Estimated Std Err:  0.019098 
              p-value:  <2e-16 
    95% Conf Interval: (0.54345, 0.61831) 

Control Estimate:
   Parameter Estimate:  0.60433 
    Estimated Std Err:  0.027291 
              p-value:  <2e-16 
    95% Conf Interval: (0.55084, 0.65782) 

Additive Treatment Effect:
   Parameter Estimate:  -0.023449 
    Estimated Std Err:  0.032411 
              p-value:  0.46937 
    95% Conf Interval: (-0.086973, 0.040075) 

Relative Risk:
   Parameter Estimate:  0.9612 
  Est Std Err log(RR):  0.054333 
              p-value:  0.46638 
    95% Conf Interval: (0.8641, 1.0692) 

Odds Ratio:
   Parameter Estimate:  0.90742 
  Est Std Err lo

To access the fit of three algorithms, we can check out the Risk and Coefficient of each at each exposure A as follows:

In [7]:
results.ltmle$fit$g[[1]]

,Risk,Coef
SL.mean_All,0.2501551,0.3468718
SL.glm_All,0.2500608,0.6531282
SL.xgboost_All,0.2710148,0.0000000
,Risk,Coef
SL.mean_All,0.1883605,0.00286270
SL.glm_All,0.1295092,0.97198324
SL.xgboost_All,0.1467643,0.02515406
,Risk,Coef
SL.mean_All,0.05741619,0.04181446
SL.glm_All,0.05590256,0.83478290


We can also access the goodness of fit of each algorithm by checking the risk and coefficient of expectation of Y given A and W, i.e. $\bar{Q}(A,W)$

In [8]:
results.ltmle$fit$Q

,Risk,Coef
SL.mean_All,0.02535072,0.005126427
SL.glm_All,0.01596320,0.727646465
SL.xgboost_All,0.01672129,0.267227108
,Risk,Coef
SL.mean_All,0.04609619,0.01851101
SL.glm_All,0.03241319,0.64381484
SL.xgboost_All,0.03420115,0.33767416
,Risk,Coef
SL.mean_All,0.07963410,0.02371482
SL.glm_All,0.05562133,0.68011898


### 2

Let us assume you have estimated counterfactual means for all 32 possible regimes, $\bar{a}$, over 5 time points and you have summarized the counterfactual means via the use of a marginal structural model (MSM): 
$$
\underset{\alpha, \beta}{argmin}\sum_{\bar{a}}(\mathbb{E}Y(5)_{\bar{a}} - (\alpha + \beta sum(\bar{a})))^2
$$

where $sum(\bar{a})$ = sum of treatments over the 5 time points in the study.  Notice, this is just a least squares projection of the counterfactual means onto a line.  How would you interpret this coefficient to a layperson, ie someone not technical?



1) The coefficient describes the causal effect of treatment over 5 time points on the outcome (dead or not), assuming the relationship between the covariates (treatment + confounders) and outcome is linear.

2) The coefficient is optimized when the fitted line is closest to the expected value, and the expected outcome is supported by all 32 possible regimes $\bar{a}$.

3) The line is fitted using Marginal Structural Model, which constructs a time ordering of treatments and other covariates (known as confounder, which might has associations with treatments and/or outcome). With the time ordering, the treatments are seperated from confounders. After seperating and controling for confounders, we can get the unboased estimates of the causal effect of treatment on outcome.

4) If the coefficient is positive, the causal effect on treatment on outcome is positive, vice versa. The larger the absolute value of coeffcient, the stronger the causal effect (whether positive or negative).

### 3

Continuing from the previous question, what if you had some regimes that were much more common than others in the data and you decided to offer estimates of the weighted projection:
$$
\underset{\alpha, \beta}{argmin}\sum_{\bar{a}}P_n(\bar{A}=\bar{a})(\mathbb{E}Y(5)_{\bar{a}} - (\alpha + \beta sum(\bar{a})))^2
$$

How would you interpret this coefficient to a layperson, ie someone not technical? $P_n(\bar{A}=\bar{a})$ is just the empirical probability of observing $\bar{a}$. Also, in what situation does this change the parameter defined in the previous question? Explain briefly.    

**Explanation**

1) The coefficient describes the causal effect of treatment over 5 time points on outcome, assuming the relationship between the covariates and outcome is linear. 

2) The fitting of model and estimates of coefficients are based on observation of data. The more common regimes are weighted more because they are more often to be observed. Similarly, the rare regimes are weighted less. The sum of weights of all 23 regimes sum up to 1, which is just a way of standard scaling.

This coefficient is the estimated line that describes the change in the probability of our outcome (survival or death) caused by observing differing levels of treatment (or exposure) over five time points, summarizing how the observed treatment regimes affect the outcome. We can think of this line as an estimated relationship between our treatment and outcome where we also allow the commonality of the treatment to affect this treatment-outcome relationship. This line is estimated similarly to the previous section but, contrary to the previous coefficient, this line is weighted by the probability of observing a treatment regime. This type of weighting can be useful when some regimes are much more common than others.
Like the previous section, we can still interpret this coefficient/line in terms of a rate (where the rate is just the slope of this line). However, the rate here is influenced by how many people got the treatment regimes.
Collaborators & Resources